In [10]:
import time
import warnings
import pandas as pd
import xgboost as xgb
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler
from feature_extraction import Feature_extraction

In [13]:
FEATURES = [
    'flow_duration', 'Header_Length', 'Protocol Type', 'Duration',
       'Rate', 'Srate', 'Drate', 'fin_flag_number', 'syn_flag_number',
       'rst_flag_number', 'psh_flag_number', 'ack_flag_number',
       'ece_flag_number', 'cwr_flag_number', 'ack_count',
       'syn_count', 'fin_count', 'urg_count', 'rst_count', 
    'HTTP', 'HTTPS', 'DNS', 'Telnet', 'SMTP', 'SSH', 'IRC', 'TCP',
       'UDP', 'DHCP', 'ARP', 'ICMP', 'IPv', 'LLC', 'Tot sum', 'Min',
       'Max', 'AVG', 'Std', 'Tot size', 'IAT', 'Number', 'Magnitue',
       'Radius', 'Covariance', 'Variance', 'Weight', 
]

LABELS = 'label'

ATTACKS = {
    'DDoS-RSTFINFlood': 0,
    'DDoS-PSHACK_Flood': 1,
    'DDoS-SYN_Flood': 2,
    'DDoS-UDP_Flood': 3,
    'DDoS-TCP_Flood': 4,
    'DDoS-ICMP_Flood': 5,
    'DDoS-SynonymousIP_Flood': 6,
    'DDoS-ACK_Fragmentation': 7,
    'DDoS-UDP_Fragmentation': 8,
    'DDoS-ICMP_Fragmentation': 9,
    'DDoS-SlowLoris': 10,
    'DDoS-HTTP_Flood': 11,
    'DoS-UDP_Flood': 12,
    'DoS-SYN_Flood': 13,
    'DoS-TCP_Flood': 14,
    'DoS-HTTP_Flood': 15,
    'Mirai-greeth_flood': 16,
    'Mirai-greip_flood': 17,
    'Mirai-udpplain': 18,
    'Recon-PingSweep': 19,
    'Recon-OSScan': 20,
    'Recon-PortScan': 21,
    'VulnerabilityScan': 22,
    'Recon-HostDiscovery': 23,
    'DNS_Spoofing': 24,
    'MITM-ArpSpoofing': 25,
    'BenignTraffic': 26,
    'BrowserHijacking': 27,
    'Backdoor_Malware': 28,
    'XSS': 29,
    'Uploading_Attack': 30,
    'SqlInjection': 31,
    'CommandInjection': 32,
    'DictionaryBruteForce': 33
}

In [2]:
start = time.time()
pcapfiles = [
    'sniffed.pcap',
]
subfiles_size = 10 # MB
split_directory = 'split_temp/'
destination_directory = 'output/'
converted_csv_files_directory = 'csv_files/'
n_threads = 8

address = "./"


for i in range(len(pcapfiles)):
    lstart = time.time()
    pcap_file = pcapfiles[i]
    fe = Feature_extraction()
    df = fe.pcap_evaluation(pcap_file,destination_directory + pcap_file)
    print(f'done! ({pcap_file})(' + str(round(time.time()-lstart, 2))+ 's)')
    
end = time.time()
print(f'Elapsed Time = {(end-start)}s')
df

done! (sniffed.pcap)(0.02s)
Elapsed Time = 0.01974940299987793s


,ts,Protocol_name,Duration,Protocol Type,flow_duration,Header_Length,src_ip_bytes,fin_flag_number,syn_flag_number,rst_flag_number,...,Variance,Weight,Correlation,RARP,DS status,Fragments,Sequence number,Protocol Version,flow_idle_time,flow_active_time
0,1.700967e+09,TCP,112,6,0.000000,66,1,0,0,0,...,0,1,0.0,0,0,0,0,0,1.700967e+09,0.000000
1,1.700967e+09,TCP,64,6,0.000000,66,1,0,0,0,...,0,4,NaN,0,0,0,0,0,1.700967e+09,0.000000
2,1.700967e+09,TCP,57,6,0.005566,132,1,0,0,0,...,0,9,NaN,0,0,0,0,0,5.565882e-03,0.005566
3,1.700967e+09,TCP,57,6,0.046599,198,2,0,0,0,...,0,16,NaN,0,0,0,0,0,4.103303e-02,0.046599
4,1.700967e+09,TCP,64,6,0.046635,264,2,0,0,0,...,0,25,NaN,0,0,0,0,0,3.600121e-05,0.046635


In [9]:
# Preprocessing Functions
scaler = StandardScaler()
scaler.fit(df[FEATURES])
df[FEATURES] = scaler.transform(df[FEATURES])
df[FEATURES].head()

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,AVG,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight
0,-0.897257,-1.028992,0.0,1.977299,-0.765012,-0.765012,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,-1.414214,0.0,0.0,0.0,0.0,-1.156243
1,-0.897257,-1.028992,0.0,-0.326350,-0.765012,-0.765012,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-0.5,-0.707107,0.0,0.0,0.0,0.0,-0.809370
2,-0.644522,-0.171499,0.0,-0.662299,1.932750,1.932750,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-0.5,0.000000,0.0,0.0,0.0,0.0,-0.231249
3,1.218701,0.685994,0.0,-0.662299,-0.281671,-0.281671,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-0.5,0.707107,0.0,0.0,0.0,0.0,0.578122
4,1.220336,1.543487,0.0,-0.326350,-0.121055,-0.121055,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-0.5,1.414214,0.0,0.0,0.0,0.0,1.618741


# Predict with our model

In [16]:
xgb_model = xgb.XGBClassifier()
xgb_model.load_model('../models/xgb_model.json')

In [14]:
list(ATTACKS)

['DDoS-RSTFINFlood',
 'DDoS-PSHACK_Flood',
 'DDoS-SYN_Flood',
 'DDoS-UDP_Flood',
 'DDoS-TCP_Flood',
 'DDoS-ICMP_Flood',
 'DDoS-SynonymousIP_Flood',
 'DDoS-ACK_Fragmentation',
 'DDoS-UDP_Fragmentation',
 'DDoS-ICMP_Fragmentation',
 'DDoS-SlowLoris',
 'DDoS-HTTP_Flood',
 'DoS-UDP_Flood',
 'DoS-SYN_Flood',
 'DoS-TCP_Flood',
 'DoS-HTTP_Flood',
 'Mirai-greeth_flood',
 'Mirai-greip_flood',
 'Mirai-udpplain',
 'Recon-PingSweep',
 'Recon-OSScan',
 'Recon-PortScan',
 'VulnerabilityScan',
 'Recon-HostDiscovery',
 'DNS_Spoofing',
 'MITM-ArpSpoofing',
 'BenignTraffic',
 'BrowserHijacking',
 'Backdoor_Malware',
 'XSS',
 'Uploading_Attack',
 'SqlInjection',
 'CommandInjection',
 'DictionaryBruteForce']

In [17]:
preds = xgb_model.predict(df[FEATURES])
[list(ATTACKS)[i] for i in preds]

['DDoS-ICMP_Flood',
 'DDoS-ICMP_Flood',
 'DDoS-ICMP_Flood',
 'DDoS-ICMP_Flood',
 'DDoS-ICMP_Flood']